In [1]:
import pandas as pd
import seaborn as sns

In [2]:
CPS_df = pd.read_csv("../data/processed/CPS_2009_2017_clean.csv")
ACS_df = pd.read_csv("../data/processed/ACS_2011_2017_clean.csv")
NFCS_df = pd.read_csv("../data/processed/NFCS_2009_2018_clean.csv")

C:\Users\Desmond\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (3,4,5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
frames = [CPS_df, ACS_df, NFCS_df]

In [4]:
#declaring STATE list
STATES = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
          "Connecticut","Delaware","District of Columbia", "Florida","Georgia","Hawaii",
          "Idaho","Illinois", "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine",
          "Maryland","Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
          "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
          "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
          "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
          "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]

In [5]:
#generating state:state_number dictionary
STATE_FIPS = list(frames[0].STATEFIP.unique())

STATE = {}
for state, name in zip(STATE_FIPS, STATES):
    STATE[state] = name

In [6]:
#generating STATE column for pertinent dfs
CPS_df["STATE"] = CPS_df.STATEFIP.map(STATE)
ACS_df["STATE"] = ACS_df.STATEFIP.map(STATE)

# Aggregatting CPS Data

In [7]:
pop_prop = pd.read_csv("../data/interim/population_proportions")
pop_prop.head()

,STATEFIP,YEAR,MONTH,REGION,COUNTY,AGE,SEX,RACE,BUNBANKED,BACCTEV,...,CINETHH,CINETHP,CINTUSE,CINTOUT,CINTOTHH,CINTLIBH,CINTSCH,CINTSCHH,CINONET,CIHSNET
0,1,2009,2954,2954,2954,2954,2954,2954,1451,1451,...,1503,0,1503,0,0,0,0,0,0,1503
1,1,2010,1553,1553,1553,1553,1553,1553,0,0,...,1553,0,0,0,0,1553,0,1553,1553,1553
2,1,2011,2913,2913,2913,2913,2913,2913,1477,1477,...,0,1436,0,0,1436,0,1436,0,1436,1436
3,1,2012,1388,1388,1388,1388,1388,1388,0,0,...,0,1388,0,0,0,1388,0,1388,1388,1388
4,1,2013,2769,2769,2769,2769,2769,2769,1412,1412,...,0,1357,0,1357,1357,0,1357,0,1357,1357


In [8]:
pop_prop = pop_prop[["YEAR", "BUNBANKED", "STATEFIP"]]

pop_prop

,YEAR,BUNBANKED,STATEFIP
0,2009,1451,1
1,2010,0,1
2,2011,1477,1
3,2012,0,1
4,2013,1412,1
...,...,...,...
352,2011,1710,56
353,2012,0,56
354,2013,1674,56
355,2015,1849,56


In [9]:
state_year_agg = []

for year in pop_prop.YEAR.unique():
    holder = pop_prop[pop_prop.YEAR == year]
    state_year_agg.append(holder)
    #national_agg_sums = [pop_prop[pop_prop.STATEFIP == state].BUNBANKED.sum() for state in pop_prop.STATEFIP.unique()]
    #print(f"{year}")
    #display(holder)

In [10]:
state_survey_pop_agg = pd.concat(state_year_agg)

In [11]:
state_survey_pop_agg["STATE"] = state_survey_pop_agg.STATEFIP.map(STATE)

In [12]:
state_survey_pop_agg

,YEAR,BUNBANKED,STATEFIP,STATE
0,2009,1451,1,Alabama
7,2009,1802,2,Alaska
14,2009,1631,4,Arizona
21,2009,1502,5,Arkansas
28,2009,11443,6,California
...,...,...,...,...
328,2017,2255,51,Virginia
335,2017,2311,53,Washington
342,2017,2692,54,West Virginia
349,2017,1865,55,Wisconsin


In [13]:
state_survey_pop_agg.rename(columns = {"BUNBANKED": "SURVEY_POP"}, inplace = True)

In [14]:
state_survey_pop_agg

,YEAR,SURVEY_POP,STATEFIP,STATE
0,2009,1451,1,Alabama
7,2009,1802,2,Alaska
14,2009,1631,4,Arizona
21,2009,1502,5,Arkansas
28,2009,11443,6,California
...,...,...,...,...
328,2017,2255,51,Virginia
335,2017,2311,53,Washington
342,2017,2692,54,West Virginia
349,2017,1865,55,Wisconsin


In [15]:
CPS_agg = pd.DataFrame()
CPS_agg["STATE"] = CPS_df.STATE
CPS_agg["UNDERBANKED"] = CPS_df.BUNBANKED
CPS_agg["YEAR"] = CPS_df.YEAR

#copying aggregation before grouping for additional breakdowns
CPS_reason_agg = CPS_agg.copy(deep=True)

CPS_agg = CPS_agg.groupby(["YEAR", "STATE"]).count()

CPS_agg = CPS_agg.reset_index()

CPS_agg

,YEAR,STATE,UNDERBANKED
0,2011,Alabama,1010
1,2011,Alaska,1377
2,2011,Arizona,1284
3,2011,Arkansas,1031
4,2011,California,8344
...,...,...,...
199,2017,Virginia,1512
200,2017,Washington,1673
201,2017,West Virginia,1465
202,2017,Wisconsin,1374


In [16]:
state_survey_pop_agg = state_survey_pop_agg[state_survey_pop_agg.YEAR.isin(CPS_agg.YEAR.unique())].reset_index()

state_survey_pop_agg

,index,YEAR,SURVEY_POP,STATEFIP,STATE
0,2,2011,1477,1,Alabama
1,9,2011,1717,2,Alaska
2,16,2011,1669,4,Arizona
3,23,2011,1378,5,Arkansas
4,30,2011,11408,6,California
...,...,...,...,...,...
199,328,2017,2255,51,Virginia
200,335,2017,2311,53,Washington
201,342,2017,2692,54,West Virginia
202,349,2017,1865,55,Wisconsin


In [17]:
CPS_agg["SURVEY_POP"] = state_survey_pop_agg.SURVEY_POP

In [18]:
CPS_agg

,YEAR,STATE,UNDERBANKED,SURVEY_POP
0,2011,Alabama,1010,1477
1,2011,Alaska,1377,1717
2,2011,Arizona,1284,1669
3,2011,Arkansas,1031,1378
4,2011,California,8344,11408
...,...,...,...,...
199,2017,Virginia,1512,2255
200,2017,Washington,1673,2311
201,2017,West Virginia,1465,2692
202,2017,Wisconsin,1374,1865


In [19]:
CPS_agg.to_csv("../data/processed/Dashboard_Data/CPS_STATE_Aggregate.csv")

In [20]:
#Isolating the specific northwest while 
PNW = ["Washington", "Oregon", "Wyoming", "Montana", "Idaho"]

In [21]:
PNW_CPS_agg = CPS_agg[CPS_agg.STATE.isin(PNW)]

PNW_CPS_agg

,YEAR,STATE,UNDERBANKED,SURVEY_POP
12,2011,Idaho,1320,1562
26,2011,Montana,1013,1230
37,2011,Oregon,1509,1876
47,2011,Washington,1980,2377
50,2011,Wyoming,1398,1710
63,2013,Idaho,1111,1464
77,2013,Montana,969,1329
88,2013,Oregon,1454,1898
98,2013,Washington,1782,2410
101,2013,Wyoming,1210,1674


In [22]:
PNW_CPS_agg.to_csv("../data/processed/Dashboard_Data/CPS_PNW_STATE_Aggregate.csv")

----------------------------------------------------------------------------------------------

# Aggregatting ACS Data

In [23]:
#ACS_df = pd.read_csv("../data/processed/ACS_2011_2017_clean")
#ACS_df["STATE"] = ACS_df.STATEFIP.map(STATE)

In [24]:
ACS_df.head()

,Unnamed: 0,YEAR,SAMPLE,REGION,STATEFIP,COUNTYFIP,HHINCOME,SEX,AGE,MARST,...,CILAPTOP,CISMRTPHN,CITABLET,CIHAND,CIHISPEED,CISAT,CIDIAL,CIOTHSVC,FAMSIZE,STATE
0,3061692,2011,201101,32,1,125,9999999,2,19,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Alabama
1,3061693,2011,201101,32,1,0,9999999,2,86,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Alabama
2,3061694,2011,201101,32,1,0,87200,1,78,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,Alabama
3,3061695,2011,201101,32,1,0,87200,2,74,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,Alabama
4,3061696,2011,201101,32,1,0,43200,1,57,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Alabama


In [25]:
ACS_df = ACS_df.drop(columns = ['Unnamed: 0'])

In [26]:
filtering_columns = ACS_df.columns

In [27]:
filtering_columns = filtering_columns.drop(["STATE", "YEAR", "SAMPLE", "REGION", 'STATEFIP'])

In [28]:
filtering_columns

Index(['COUNTYFIP', 'HHINCOME', 'SEX', 'AGE', 'MARST', 'RACE', 'CITIZEN',
       'EDUC', 'OCC2010', 'CINETHH', 'CILAPTOP', 'CISMRTPHN', 'CITABLET',
       'CIHAND', 'CIHISPEED', 'CISAT', 'CIDIAL', 'CIOTHSVC', 'FAMSIZE'],
      dtype='object')

In [29]:
pivot_df = ACS_df.copy(deep=True)
#using filter to generate multiple pivot tables for data vizualization
for _filter in filtering_columns:
    pivot_df[f"{_filter}_COUNTS"] = pivot_df[_filter]
    pivot_df_final = pivot_df[["YEAR", "REGION", "STATE", _filter, f"{_filter}_COUNTS"]].groupby(["YEAR", "REGION", "STATE", _filter]).count()
    #display(pivot_df[["YEAR", "REGION", "STATE", _filter, f"{_filter}_COUNTS"]].groupby(["YEAR", "REGION", "STATE", _filter]).count())
    #display(pivot_df_final)
    pivot_df_final.to_csv(f"../data/processed/Dashboard_Data/{_filter}_ACS_AGG.csv")

In [30]:
ACS_df.groupby(["YEAR", "REGION", "STATE", "CINETHH"]).count()#.value_counts()

SAMPLE  STATEFIP  COUNTYFIP  HHINCOME    SEX  \
YEAR REGION STATE       CINETHH                                                 
2013 11     Connecticut 0.0        2257      2257       2257      2257   2257   
                        1.0       29482     29482      29482     29482  29482   
                        2.0         965       965        965       965    965   
                        3.0        3678      3678       3678      3678   3678   
            Maine       0.0         679       679        679       679    679   
...                                 ...       ...        ...       ...    ...   
2018 42     Oregon      3.0        2140      2140       2140      2140   2140   
            Washington  0.0        3242      3242       3242      3242   3242   
                        1.0       68293     68293      68293     68293  68293   
                        2.0        1274      1274       1274      1274   1274   
                        3.0        3416      3416       3416      3416   3416   

                                   AGE  MARST   RACE  CITIZEN   EDUC  OCC2010  \
YEAR REGION STATE       CINETHH                                                 
2013 11     Connecticut 0.0       2257   2257   2257     2257   2257     2257   
                        1.0      29482  29482  29482    29482  29482    29482   
                        2.0        965    965    965      965    965      965   
                        3.0       3678   3678   3678     3678   3678     3678   
            Maine       0.0        679    679    679      679    679      679   
...                                ...    ...    ...      ...    ...      ...   
2018 42     Oregon      3.0       2140   2140   2140     2140   2140     2140   
            Washington  0.0       3242   3242   3242     3242   3242     3242   
                        1.0      68293  68293  68293    68293  68293    68293   
                        2.0       1274   1274   1274     1274   1274     1274   
                        3.0       3416   3416   3416     3416   3416     3416   

                                 CILAPTOP  CISMRTPHN  CITABLET  CIHAND  \
YEAR REGION STATE       CINETHH                                          
2013 11     Connecticut 0.0          2257          0         0    2257   
                        1.0         29482          0         0   29482   
                        2.0           965          0         0     965   
                        3.0          3678          0         0    3678   
            Maine       0.0           679          0         0     679   
...                                   ...        ...       ...     ...   
2018 42     Oregon      3.0          2140       2140      2140       0   
            Washington  0.0          3242       3242      3242       0   
                        1.0         68293      68293     68293       0   
                        2.0          1274       1274      1274       0   
                        3.0          3416       3416      3416       0   

                                 CIHISPEED  CISAT  CIDIAL  CIOTHSVC  FAMSIZE  
YEAR REGION STATE       CINETHH                                               
2013 11     Connecticut 0.0           2257   2257    2257      2257     2257  
                        1.0          29482  29482   29482     29482    29482  
                        2.0            965    965     965       965      965  
                        3.0           3678   3678    3678      3678     3678  
            Maine       0.0            679    679     679       679      679  
...                                    ...    ...     ...       ...      ...  
2018 42     Oregon      3.0           2140   2140    2140      2140     2140  
            Washington  0.0           3242   3242    3242      3242     3242  
                        1.0          68293  68293   68293     68293    68293  
                        2.0           1274   1274    1274      1274     1274  
              

In [31]:
ACS_df.columns

Index(['YEAR', 'SAMPLE', 'REGION', 'STATEFIP', 'COUNTYFIP', 'HHINCOME', 'SEX',
       'AGE', 'MARST', 'RACE', 'CITIZEN', 'EDUC', 'OCC2010', 'CINETHH',
       'CILAPTOP', 'CISMRTPHN', 'CITABLET', 'CIHAND', 'CIHISPEED', 'CISAT',
       'CIDIAL', 'CIOTHSVC', 'FAMSIZE', 'STATE'],
      dtype='object')

* HHINCOME = House Hold Income
* MARST = Marital Status
* OCC2010 = Occupation
* CINETHH = Access to Internet
* CILAPTOP = Laptop, desktop, or notebook computer
* CISMRTPHN = Smartphone
* CITABLET = Tablet or other portable wireless computer
* CIHAND = Handheld Computer
* CIHISPEED = Broadband (high speed) Internet service such as cable, fiber optic, or DSL service
* CISAT = Satellite internet service
* CIDIAL = Dial-up Service
* CIOTHSVC = Other Internet Service

In [32]:
ACS_agg = pd.DataFrame()
ACS_agg["STATE"] = ACS_df.STATE
ACS_agg["OCC2010"] = ACS_df.OCC2010
ACS_agg["CINETHH"] = ACS_df.CINETHH
ACS_agg["CILAPTOP"] = ACS_df.CILAPTOP
ACS_agg["CISMRTPHN"] = ACS_df.CISMRTPHN
ACS_agg["CITABLET"] = ACS_df.CITABLET
ACS_agg["CIHAND"] = ACS_df.CIHAND
ACS_agg["CIHISPEED"] = ACS_df.CIHISPEED
ACS_agg["CISAT"] = ACS_df.CISAT
ACS_agg["CIDIAL"] = ACS_df.CIDIAL
ACS_agg["CIOTHSVC"] = ACS_df.CIOTHSVC
ACS_agg["YEAR"] = ACS_df.YEAR

In [33]:
ACS_agg = ACS_agg.groupby(["STATE", "YEAR"]).count()

In [34]:
ACS_agg = ACS_agg.reset_index()

ACS_agg

,STATE,YEAR,OCC2010,CINETHH,CILAPTOP,CISMRTPHN,CITABLET,CIHAND,CIHISPEED,CISAT,CIDIAL,CIOTHSVC
0,Alabama,2011,48579,0,0,0,0,0,0,0,0,0
1,Alabama,2012,47819,0,0,0,0,0,0,0,0,0
2,Alabama,2013,47836,47836,47836,0,0,47836,47836,47836,47836,47836
3,Alabama,2015,47476,47476,47476,0,0,47476,47476,47476,47476,47476
4,Alabama,2017,47645,47645,47645,47645,47645,0,47645,47645,47645,47645
...,...,...,...,...,...,...,...,...,...,...,...,...
301,Wyoming,2012,5824,0,0,0,0,0,0,0,0,0
302,Wyoming,2013,5828,5828,5828,0,0,5828,5828,5828,5828,5828
303,Wyoming,2015,5819,5819,5819,0,0,5819,5819,5819,5819,5819
304,Wyoming,2017,5912,5912,5912,5912,5912,0,5912,5912,5912,5912


In [35]:
ACS_agg.to_csv("../data/processed/Dashboard_Data/ACS_STATE_Aggregate.csv")

----------------------------------------------------------------------------------------------

# Aggregating NFCS

In [36]:
NFCS_df.head()

,Unnamed: 0,STATE,CHECKING_ACCOUNT,AUTO_TITLE_LOAN,PAYDAY_LOAN,PAWN_SHOP,RENT_TO_OWN,YEAR
0,0,24,1,1,1,1,1,2012
1,1,10,1,1,1,1,1,2012
2,2,23,1,1,1,1,1,2012
3,3,14,1,1,1,1,1,2012
4,4,44,1,1,2,2,5,2012


In [37]:
NFCS_df.drop("Unnamed: 0", axis=1,inplace=True)

In [38]:
#declaring STATE list
STATES = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
          "Connecticut","Delaware","District of Columbia", "Florida","Georgia","Hawaii",
          "Idaho","Illinois", "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine",
          "Maryland","Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
          "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
          "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
          "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
          "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]

In [39]:
#generating state:state_number dictionary
STATE_NFCS = list(NFCS_df.STATE.unique())
STATE_NFCS.sort()

STATE = {}
for state, name in zip(STATE_NFCS, STATES):
    STATE[state] = name

In [40]:
NFCS_df.STATE = NFCS_df.STATE.map(STATE)

In [41]:
NFCS_df.STATE

0         Minnesota
1           Florida
2          Michigan
3          Illinois
4             Texas
            ...    
133814      Wyoming
133815        Maine
133816        Maine
133817        Maine
133818        Maine
Name: STATE, Length: 133819, dtype: object

In [42]:
NFCS_agg = NFCS_df.groupby(["STATE", "YEAR"]).count()

NFCS_agg

CHECKING_ACCOUNT  AUTO_TITLE_LOAN  PAYDAY_LOAN  PAWN_SHOP  \
STATE     YEAR                                                              
Alabama   2012               500              500          500        500   
          2015              1560             1560         1560       1560   
          2018               501              501          501        501   
Alaska    2012               501              501          501        501   
          2015              1426             1426         1426       1426   
...                          ...              ...          ...        ...   
Wisconsin 2015              1554             1554         1554       1554   
          2018               506              506          506        506   
Wyoming   2012               500              500          500        500   
          2015              1431             1431         1431       1431   
          2018               501              501          501        501   

                RENT_TO_OWN  
STATE     YEAR               
Alabama   2012          500  
          2015         1560  
          2018          501  
Alaska    2012          501  
          2015         1426  
...                     ...  
Wisconsin 2015         1554  
          2018          506  
Wyoming   2012          500  
          2015         1431  
          2018          501  

[153 rows x 5 columns]

In [43]:
NFCS_agg.to_csv("../data/processed/Dashboard_Data/NFCS_STATE_Aggregate.csv")